In [1]:
import lxml.html as parser
import requests
import csv
from urllib.parse import urljoin

In [2]:
r = requests.get("https://pt.wikipedia.org/wiki/Clube_Atl%C3%A9tico_Mineiro")
html = parser.fromstring(r.text)

## Related Links

We start by finding all the page related links that are from wikipedia and nothing else.

Preliminar investigations show:
1. Links are relative
2. Links start with /wiki/{Some other article}
3. All related links are within a div with id=bodyContent
4. There are no semicolon in the links (like this bad link '/wiki/Ficheiro:Atletico_mineiro_galo.png')

With that in mind we may start building our xpath expression

In [8]:
# let's get all links inside this div

links = html.xpath('//div[@id="bodyContent"]/descendant::a/@href')
print(links[0])
print(len(links))

#mw-head
3362


So we have 3362 links in this page within bodyContent

Let's now improve the xpath expression with some regex to filter links that do not start with /wiki/ and have semicolons in them.

In [36]:
articles = html.xpath('//div[@id="bodyContent"]/descendant::a[re:test(@href, "^/wiki/[^:]*$")]/@href', 
        namespaces={"re": "http://exslt.org/regular-expressions"})
print(len(articles))

2316


Now we just need to prepare these links and get rid of duplicates

In [41]:
base_url = "https://pt.wikipedia.org/"
articles = [urljoin(base_url, a) for a in articles]
articles = list(set(articles))
print(len(articles))
print(articles[0])

1205
https://pt.wikipedia.org/wiki/Zamora_F%C3%BAtbol_Club


## Item extraction

We must extract a few things from each page, the title, the first paragraph and the main image.
By inspecting the page we have that:

1. The title is at a h1 tag with id=firstHeading
2. First paragraph is the first p inside a div with class mw-parser-output
3. The main image is the first image inside the table inside the div with class mw-parser-output

In [43]:
title = html.xpath('//h1[@id="firstHeading"]/text()')[0]
print(title)

Clube Atlético Mineiro


In [48]:
paragraph = html.xpath('//div[@class="mw-parser-output"]/p[1]/descendant-or-self::*/text()')
print(paragraph)

['O ', 'Clube Atlético Mineiro', ' (conhecido apenas por ', 'Atlético', ' e cujo ', 'acrônimo', ' é ', 'CAM', ') é um clube brasileiro de futebol sediado na cidade de ', 'Belo Horizonte', ', ', 'Minas Gerais', '.', '[', '4', ']', ' Fundado em 25 de março de 1908 por um grupo de estudantes, tem como suas cores tradicionais o preto e o branco. Contudo, o clube teve como primeiro nome ', 'Athlético Mineiro Football Club', ' .', '[', '5', ']', ' Seu símbolo e alcunha mais popular é o Galo, mascote oficial no final da década de 1930.', '[', '6', ']', '[', '7', ']', ' O Atlético é um dos ', 'clubes mais populares do Brasil', '.', '[', '8', ']', '[', '9', ']']


In [49]:
# We need to join the result
paragraph = "".join(html.xpath('//div[@class="mw-parser-output"]/p[1]/descendant-or-self::*/text()'))
print(paragraph)

O Clube Atlético Mineiro (conhecido apenas por Atlético e cujo acrônimo é CAM) é um clube brasileiro de futebol sediado na cidade de Belo Horizonte, Minas Gerais.[4] Fundado em 25 de março de 1908 por um grupo de estudantes, tem como suas cores tradicionais o preto e o branco. Contudo, o clube teve como primeiro nome Athlético Mineiro Football Club .[5] Seu símbolo e alcunha mais popular é o Galo, mascote oficial no final da década de 1930.[6][7] O Atlético é um dos clubes mais populares do Brasil.[8][9]


In [56]:
# Better but we don't want the citations
paragraph = "".join(html.xpath('//div[@class="mw-parser-output"]/p[1]/' \
                               'descendant-or-self::*[self::a[not(starts-with(@href, "#cite"))] ' \
                               'or self::b or self::p]/text()'))
print(paragraph)

O Clube Atlético Mineiro (conhecido apenas por Atlético e cujo acrônimo é CAM) é um clube brasileiro de futebol sediado na cidade de Belo Horizonte, Minas Gerais. Fundado em 25 de março de 1908 por um grupo de estudantes, tem como suas cores tradicionais o preto e o branco. Contudo, o clube teve como primeiro nome  . Seu símbolo e alcunha mais popular é o Galo, mascote oficial no final da década de 1930. O Atlético é um dos clubes mais populares do Brasil.


Here we are selecting all the children that are either the paragraph it self, a b tag or a anchor that it's href do not start with #cite(the citations)

In [63]:
image = html.xpath('//div[@class="mw-parser-output"]/table/descendant::img/@src')[0]
print(image)

//upload.wikimedia.org/wikipedia/commons/thumb/5/5f/Atletico_mineiro_galo.png/120px-Atletico_mineiro_galo.png
